# **Τεχνητή Νοημοσύνη 2**
#**Εργασία 4b**
Στέφανος Μπακλαβάς 1115201700093

In [ ]:

from google.colab import drive
drive.mount("/content/drive")



import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error


!pip install beautifulsoup4
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import numpy as np
from datetime import datetime

from bs4 import BeautifulSoup

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_predict
from sklearn.naive_bayes import MultinomialNB



from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.metrics import f1_score



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install datasets transformers
!pip install torchvision 
!pip install sentencepiece

import torch
import torchvision


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


## Utilizing Gpu if available


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
from datasets import load_dataset

datasets = load_dataset("squad_v2")


Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
datasets
train_data = datasets['train']
validation_data = datasets['validation']

In [ ]:
train_data[1]


{'answers': {'answer_start': [207], 'text': ['singing and dancing']},
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'id': '56be85543aeaaa14008c9065',
 'question': 'What areas did Beyonce compete in when she was growing up?',
 'title': 'Beyoncé'}

In [ ]:
len(train_data)

130319

Θέτουμε το μέγιστο μήκος για κάθε sequence καθώς και το μέγιστο μήκος που θα έχει η επικάλυψη δύο κειμένων. Αν ένα κείμενο έχει μήκος μεγαλύτερο από 384 τότε σπάει σε μικρότερα τα οποία επικαλύπτονται

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
from transformers import AutoTokenizer,AdamW,BertForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Παρακάτω για κάθε ερώτηση που υπάρχιει για κάθε κείμενο βρίσκουμε το τέλος και την αρχή της tokenized απάντησης μέσα στο κείμενο , αφού η θέση της μπορεί να αλλάξει μετά το tokenization. Η παρακάτω συνάρτηση αποτελεί έναν από τους δύο standard τρόπους με τους οποίους γίνεται αυτό.

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"] ,
        examples["context" ],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

    #isws na metatrepsw auta se listes kai na tis epistrefw

Εφαρμογή της συνάρτησησς prepare_train_features για κάθε ερώτηση του dataset

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-2ec3ec6f1a16a4bc.arrow


  0%|          | 0/12 [00:00<?, ?ba/s]

Μεταρτροπή των δεδομένων του dataset ποου χρειαζόμαστε σε tensors και σπάσιμο σε train και validation sets

In [ ]:
train_input_ids =  torch.tensor(tokenized_datasets["train"]["input_ids"])
validation_input_ids = torch.tensor(tokenized_datasets["validation"]["input_ids"])

train_attention_mask =  torch.tensor(tokenized_datasets["train"]["attention_mask"])
validation_attention_mask = torch.tensor(tokenized_datasets["validation"]["attention_mask"])

train_start_positions =  torch.tensor(tokenized_datasets["train"]["start_positions"])
validation_start_positions = torch.tensor(tokenized_datasets["validation"]["start_positions"])

train_end_positions =  torch.tensor(tokenized_datasets["train"]["end_positions"])
validation_end_positions = torch.tensor(tokenized_datasets["validation"]["end_positions"])





Δημιουργία των τελικών μορφών του train και validation sets στην μορφή που τα χρειάζεται το μοντέλο

In [ ]:
train_tokenized_dataset = list(zip(*(train_input_ids,train_attention_mask,train_start_positions,train_end_positions)))
validation_tokenized_dataset = list(zip(*(validation_input_ids,validation_attention_mask,validation_start_positions,validation_end_positions)))

train_tokenized_dataset[2][3]

#train_tokenized_dataset = train_tokenized_dataset[:4]
#validation_tokenized_dataset = validation_tokenized_dataset[:4]

tensor(143)

In [ ]:
tokenized_datasets["train"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 131754
})

Χρησιμοποιείται ως model το distilbert-base-uncased του DistilBertForQuestionAnswering , ενώ το learning rate με τα καλύτερα αποτελέσματα ήταν το 5e-5. Πρέπει να σημειωθεί ότι και τα τρια learning rates που χρησιμοποιήθηκαν είχαν σχεδόν ίδιο αποτέλεσμα.Συγκεκριμένα έγιναν δοκιμές με lr = 2e-5, lr = 3e-5 και lr = 5e-5. Δυστυχώς λόγω περιορισμών χρήσης στην GPU δεν κατάφερα να τρέξω κάποιο μοντέλο για πάνω από μία epoch οπότε τα αποτελέσματα του κάθε learning rate στο readme είναι για μια epoch


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
# Tell pytorch to run this model on the GPU.

optimizer = torch.optim.Adam(model.parameters(),  lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


#optimizer = AdamW(model.parameters(),
 #                 lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
  #                eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
   #             )




model.cuda()


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            

Προετοιμασία των dataloaders .Το batch size που χρησιμοποιήθηκε και στις τρεις δοκιμές είναι το 24 καθώς αυτό παρουσιάζεται και στο σχετικό paper

In [ ]:



dataloader_train = DataLoader(
            train_tokenized_dataset,  # The training samples.
            shuffle = True, 
            batch_size = 24, # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dataloader_validation = DataLoader(
            validation_tokenized_dataset, # The validation samples.
            shuffle = True, 
            batch_size = 24, # Evaluate with this batch size.

        )

Συναρτήσεις για τον υπολογισμό του f1 score των batches

In [ ]:
def cal_average(num):
    sum_num = 0.0
    for t in num:
        sum_num = sum_num + t           

    avg = sum_num / len(num)
    return avg


#f1 score of one example
def compute_f1(real_start,real_end, pred_start,pred_end):
  #gold_toks = get_tokens(real_pos)
  #pred_toks = get_tokens(pred_pos)
  if real_start < pred_start:
    start = pred_start
  else:
    start = real_start
  if real_end > pred_end:
    end = pred_end
  else:
    end = real_end     
  #common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  
  num_same = end - start
  pred_same = pred_end - pred_start
  real_same = real_end - real_start
  
  if num_same < 0:
    return 0 
  if real_same == 0 or pred_same == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    if real_start == pred_start:
      return 1
    else:
      return 0  
    #return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0

  num_same  = num_same+1
  pred_same = pred_same +1
  real_Same = real_same +1

  precision = 1.0 * num_same / pred_same
  recall = 1.0 * num_same / real_same
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

#f1 score of epoch
#here each arguiment is a list
def f1_of_epoch(real_start,real_end, pred_start,pred_end):

  f1_scores = []
  length = len(real_start)
  for i in range(length):
    f1 = compute_f1(real_start[i],real_end[i],pred_start[i],pred_end[i])
    f1_scores.append(f1)
    #print("added f1 = ",f1)

  f1_final = cal_average(f1_scores)
  
  return f1_final


In [ ]:
!pip install tqdm
import tqdm

Training and validation of model 

In [ ]:
from numpy.core.fromnumeric import argmax
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

from scipy.special import softmax
import random 
import warnings
warnings.filterwarnings('ignore')

# Set the seed value .
seed_val = 5

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


def Average(lst):
    return sum(lst) / len(lst)

train_loss = []
validation_loss = []
epochs = []
fpr = dict()
tpr = dict()
roc_auc = dict()
best_epoch_preds = []
best_epoch_y_values = []
prev_max_presicion = 0

for epoch in range(4):
  
  batch_losses_train = []
  batch_losses_validation = []

  start_preds = []
  end_preds = []

  start_values = []
  end_values = []


  model.train()
  
  counter1 = 0


  for batch in dataloader_train:


      if counter1 % 100 == 0:
        print("training batch = ",counter1)
      counter1 = counter1+1

    
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      start_positions = batch[2].to(device)
      end_positions = batch[3].to(device)


      model.zero_grad()        

      output = model(b_input_ids,  
                             attention_mask=b_input_mask, 
                             start_positions = start_positions,end_positions = end_positions)



      loss = output.loss
      start_logits = output.start_logits
      end_logits = output.end_logits
      batch_losses_train.append(loss.item())
     
      loss.backward()

      optimizer.step()


  train_loss.append(Average(batch_losses_train))
  batch_losses_train.clear

  model.eval()

  counter2 = 0

  torch.save(model.state_dict(),'/content/drive/MyDrive/Colab Notebooks/model3_AI_2_HW4.pt')

  for batch in dataloader_validation:

   
      if counter2 % 100 == 0:
        print("validation batch = ",counter2)
      counter2 = counter2 +1
  
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      start_positions = batch[2].to(device)
      end_positions = batch[3].to(device)


      with torch.no_grad():

        output = model(b_input_ids,  
                              attention_mask=b_input_mask, 
                              start_positions = start_positions,end_positions = end_positions)


      loss = output.loss
      start_logits = output.start_logits
      end_logits = output.end_logits
      batch_losses_validation.append(loss.item())
     
      start_positions = start_positions.to('cpu').numpy()
      end_positions = end_positions.to('cpu').numpy()

      start_logits = start_logits.detach().cpu().numpy()
      end_logits = end_logits.detach().cpu().numpy()


      start_logits = softmax(start_logits)
      end_logits = softmax(end_logits)
      for pred in start_logits:
        start_preds.append(argmax(pred))
      for v_pred in end_logits:
        end_preds.append(argmax(v_pred))
      for val in start_positions:
        start_values.append(val) 
      for val in end_positions:
        end_values.append(val)   

      batch_losses_validation.append(loss.item())




  validation_loss.append(Average(batch_losses_validation))
  batch_losses_validation.clear

  #############################################################################################################
  #calculating scores
  epochs.append(epoch)
  validation_loss.append(Average(batch_losses_validation))
 
  print("\n LAST 30 PREDICTIONS \n")
  print("real start = ",start_values[-30:])
  print("real end = ",end_values[-30:])
  print("predicted start = ",start_preds[-30:])
  print("predicted end = ",end_preds[-30:])

  #print("start_values length = ",len(start_values))
  f1_score = f1_of_epoch(start_values,end_values,start_preds,end_preds)
  print("\n\nf1_score = ",f1_score)


  start_preds.clear()
  end_preds.clear()

  start_values.clear()
  end_values.clear()



  batch_losses_validation.clear


training batch =  0
training batch =  100
training batch =  200
training batch =  300
training batch =  400
training batch =  500
training batch =  600
training batch =  700
training batch =  800
training batch =  900
training batch =  1000
training batch =  1100
training batch =  1200
training batch =  1300
training batch =  1400
training batch =  1500
training batch =  1600
training batch =  1700
training batch =  1800
training batch =  1900
training batch =  2000
training batch =  2100
training batch =  2200
training batch =  2300
training batch =  2400
training batch =  2500
training batch =  2600
training batch =  2700
training batch =  2800
training batch =  2900
training batch =  3000
training batch =  3100
training batch =  3200
training batch =  3300
training batch =  3400
training batch =  3500
training batch =  3600
training batch =  3700
training batch =  3800
training batch =  3900
training batch =  4000
training batch =  4100
training batch =  4200
training batch =  4300
